In [91]:
import pandas as pd
import math
import os

In [26]:
wd = os.getcwd()
data_path = wd + "/data/"
for doc in os.listdir(data_path):
    print(doc)

Appendix A - Incel Lexicon.pdf
incels.is_AllComments.anon
reddit-braincels-posts.anon.txt
reddit-incel-posts.anon.txt
reddit-IncelExit-posts.anon.txt
saidit-incel-posts.anon.txt


In [123]:
incels_dataset = pd.read_csv(data_path +"reddit-braincels-posts.anon.txt", sep="\t", encoding="utf-8",
names=["link", "comment_id", "user_id", "parent", "timestamp", "title", "text"])

ParserError: Error tokenizing data. C error: Expected 7 fields in line 1541, saw 8


In [28]:
incels = incels_dataset.drop([0,1])

In [34]:
incels_reduced

,link,comment_id,user_id,parent,timestamp,title,text
2,https://www.reddit.com/r/Braincels/comments/77...,77ru0b,9.249790e+08,NaN,1508566505,Getting Settled in,General discussion and Q&amp;A thread since we...
3,/r/Braincels/comments/doqectl/comment/doqgmkg,doqgmkg,9.314841e+08,doqectl,1508701504,NaN,Thanks. It just seemed that it would be anothe...
4,/r/Braincels/comments/doqe6l5/comment/doqectl,doqectl,9.249790e+08,doqe6l5,1508699066,NaN,"To your last question, of course your safe her..."
5,/r/Braincels/comments/77ru0b/comment/doq6k7h,doq6k7h,9.314841e+08,77ru0b,1508690481,NaN,Is FHO also used here or is this like FA / IWH
6,/r/Braincels/comments/doplfy9/comment/dopnsx0,dopnsx0,9.249790e+08,doplfy9,1508652013,NaN,I think the main thing about r/IncelDiscussion...
...,...,...,...,...,...,...,...
97,/r/Braincels/comments/77rwo9/comment/du6mnst,du6mnst,9.250076e+08,77rwo9,1518536790,NaN,A million dancing dots came to form this mass\...
98,/r/Braincels/comments/77rwo9/comment/du5b2ht,du5b2ht,9.250079e+08,77rwo9,1518470279,NaN,"**One Path**\n\nLeft and right, the road stret..."
99,/r/Braincels/comments/77rwo9/comment/du4tw76,du4tw76,9.250039e+08,77rwo9,1518454033,NaN,why is sex so important to y’all\n\n&amp;nbsp;...
100,/r/Braincels/comments/77rwo9/comment/du4tulo,du4tulo,9.250039e+08,77rwo9,1518453991,NaN,why is sex so important to y’all\nsaid Stacy w...


In [67]:
chats = {}
edges = []
nodes = set()

# for index, row in incels_reduced.iterrows():
#     link, c_id, u_id, p_id, timestamp, title, text = row 
    
#     nodes.add(u_id)
    
#     if title:
#         chats[c_id] = {
#             'text' : text,
#             'user_id': u_id,
#             'childs' : []
#         }


In [61]:
incels_reduced

,link,comment_id,user_id,parent,timestamp,title,text
2,https://www.reddit.com/r/Braincels/comments/77...,77ru0b,9.249790e+08,NaN,1508566505,Getting Settled in,General discussion and Q&amp;A thread since we...
3,/r/Braincels/comments/doqectl/comment/doqgmkg,doqgmkg,9.314841e+08,doqectl,1508701504,NaN,Thanks. It just seemed that it would be anothe...
4,/r/Braincels/comments/doqe6l5/comment/doqectl,doqectl,9.249790e+08,doqe6l5,1508699066,NaN,"To your last question, of course your safe her..."
5,/r/Braincels/comments/77ru0b/comment/doq6k7h,doq6k7h,9.314841e+08,77ru0b,1508690481,NaN,Is FHO also used here or is this like FA / IWH
6,/r/Braincels/comments/doplfy9/comment/dopnsx0,dopnsx0,9.249790e+08,doplfy9,1508652013,NaN,I think the main thing about r/IncelDiscussion...
...,...,...,...,...,...,...,...
97,/r/Braincels/comments/77rwo9/comment/du6mnst,du6mnst,9.250076e+08,77rwo9,1518536790,NaN,A million dancing dots came to form this mass\...
98,/r/Braincels/comments/77rwo9/comment/du5b2ht,du5b2ht,9.250079e+08,77rwo9,1518470279,NaN,"**One Path**\n\nLeft and right, the road stret..."
99,/r/Braincels/comments/77rwo9/comment/du4tw76,du4tw76,9.250039e+08,77rwo9,1518454033,NaN,why is sex so important to y’all\n\n&amp;nbsp;...
100,/r/Braincels/comments/77rwo9/comment/du4tulo,du4tulo,9.250039e+08,77rwo9,1518453991,NaN,why is sex so important to y’all\nsaid Stacy w...


In [119]:

incels_reduced_small = incels[:100]
incels_reduced_small.fillna(False)

while not incels_reduced_small.empty:
    indices_to_drop = []
    for idx, link, c_id, u_id, p_id, timestamp, title, text in incels_reduced_small.itertuples(index=True, name=None):
    
        if type(title) == str:
            chats[c_id] = {
            'text' : text,
            'user_id': u_id,
            'childs' : []
        }
            indices_to_drop.append(idx)
            continue 
        
        if p_id in chats:
            
            chats[c_id] = {
                'text': text,
                'user_id': u_id,
                'childs': [],
                'parent' : p_id
        }   
            chats[p_id]['childs'] = chats[p_id]['childs'] + [c_id]
            edges.append((chats[p_id]['user_id'], u_id))

            indices_to_drop.append(idx)
        print(indices_to_drop)
    
    print(f"remaining rows: {len(incels_reduced_small)}")
    if not indices_to_drop:
            break   
    
    incels_reduced_small = incels_reduced_small.drop(index=indices_to_drop)
       



[2, 3]
[2, 3]
[2, 3, 5]
[2, 3, 5, 6]
[2, 3, 5, 6, 7]
[2, 3, 5, 6, 7, 8]
[2, 3, 5, 6, 7, 8, 9]
[2, 3, 5, 6, 7, 8, 9, 10]
[2, 3, 5, 6, 7, 8, 9, 10, 11]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 

In [122]:
incels_reduced

,link,comment_id,user_id,parent,timestamp,title,text
2,https://www.reddit.com/r/Braincels/comments/77...,77ru0b,9.249790e+08,NaN,1508566505,Getting Settled in,General discussion and Q&amp;A thread since we...
3,/r/Braincels/comments/doqectl/comment/doqgmkg,doqgmkg,9.314841e+08,doqectl,1508701504,NaN,Thanks. It just seemed that it would be anothe...
4,/r/Braincels/comments/doqe6l5/comment/doqectl,doqectl,9.249790e+08,doqe6l5,1508699066,NaN,"To your last question, of course your safe her..."
5,/r/Braincels/comments/77ru0b/comment/doq6k7h,doq6k7h,9.314841e+08,77ru0b,1508690481,NaN,Is FHO also used here or is this like FA / IWH
6,/r/Braincels/comments/doplfy9/comment/dopnsx0,dopnsx0,9.249790e+08,doplfy9,1508652013,NaN,I think the main thing about r/IncelDiscussion...
...,...,...,...,...,...,...,...
97,/r/Braincels/comments/77rwo9/comment/du6mnst,du6mnst,9.250076e+08,77rwo9,1518536790,NaN,A million dancing dots came to form this mass\...
98,/r/Braincels/comments/77rwo9/comment/du5b2ht,du5b2ht,9.250079e+08,77rwo9,1518470279,NaN,"**One Path**\n\nLeft and right, the road stret..."
99,/r/Braincels/comments/77rwo9/comment/du4tw76,du4tw76,9.250039e+08,77rwo9,1518454033,NaN,why is sex so important to y’all\n\n&amp;nbsp;...
100,/r/Braincels/comments/77rwo9/comment/du4tulo,du4tulo,9.250039e+08,77rwo9,1518453991,NaN,why is sex so important to y’all\nsaid Stacy w...


In [121]:
for key, val in chats.items():
    print(key)

77ru0b
doqgmkg
doqectl
doq6k7h
dopnsx0
dopm17l
doplfy9
dopk1cs
dopj9dk
dopcpxa
dopbxoz
doo7s2k
77rwdx
77rwhw
77rwo9
dx5m14k
dx45yk1
dx0rw5f
dwzw632
dwztnsd
dwzr22s
dwznki8
dwzneuj
dwznd4j
dwzkyuc
dwzktju
dwzksi5
dwzjd3y
dwyfv4e
dwpe157
dwpdt6q
dwp3jg3
dwn93v8
dwn8y4t
dwn2n6j
dwmqvnb
dwm9n1u
dwl94tm
dwkeb6w
dwke8dt
dwk6cab
dwk69rz
dwio5vv
dwetmm3
dwdoaif
dwahi9a
dw83506
dw7zt19
dw7192x
dw1p5ea
dvzjhsk
dvz7j4w
dvy6hdb
dvwq7ei
dvvll35
dvusduf
dvulpw0
dvtvfqu
dvqy087
dvqbt39
dvq0gj8
dvowddn
dvomngs
dvoc4sh
dvo523j
dvo200b
dvlxq6l
dvloizp
dvc8g08
dvbxhgd
dvb9phc
dvb80o2
dvag30l
dv9c91x
dv9ab1s
dv92ldl
dv91395
dv0b46p
dv0aut4
duyx501
duticd2
dur0p0m
duqhwmt
duou9ap
dul24b5
dukvaze
dugkbo4
duevrg2
dudqngm
duapgcd
du9dh27
du9d56i
du797mk
du6y6dw
du6y3ru
du6mnst
du5b2ht
du4tw76
du4tulo
du39lo1


In [108]:
incels_reduced

,link,comment_id,user_id,parent,timestamp,title,text
4,/r/Braincels/comments/doqe6l5/comment/doqectl,doqectl,9.249790e+08,doqe6l5,1508699066,NaN,"To your last question, of course your safe her..."
26,/r/Braincels/comments/dvhdgmx/comment/dwzkyuc,dwzkyuc,9.249832e+08,dvhdgmx,1523140390,NaN,"Yeah, the sub you’re on."
32,/r/Braincels/comments/ds48ji1/comment/dwpdt6q,dwpdt6q,9.249849e+08,ds48ji1,1522719235,NaN,If a 4 foot mongoloid with an alligator arm ga...
55,/r/Braincels/comments/dtue0ba/comment/dvwq7ei,dvwq7ei,9.249940e+08,dtue0ba,1521407070,NaN,"A part of, apart from"
72,/r/Braincels/comments/ds48ji1/comment/dvb9phc,dvb9phc,9.250006e+08,ds48ji1,1520412286,NaN,&gt;Write a girl a poem like that and she’d be...
75,/r/Braincels/comments/ds48ji1/comment/dv9c91x,dv9c91x,9.250017e+08,ds48ji1,1520329145,NaN,This is why I hate roasties.\n\nOnly chads c...
81,/r/Braincels/comments/drjipw7/comment/duyx501,duyx501,9.250039e+08,drjipw7,1519839058,NaN,unreal
83,/r/Braincels/comments/ds1hdpj/comment/dur0p0m,dur0p0m,9.250025e+08,ds1hdpj,1519459960,NaN,Why even bother trying to get women? I haven\'...
84,/r/Braincels/comments/ds1hdpj/comment/duqhwmt,duqhwmt,9.250046e+08,ds1hdpj,1519432311,NaN,I'm a tall lady; I date guys who are shorter t...
86,/r/Braincels/comments/drcbeqm/comment/dul24b5,dul24b5,9.250048e+08,drcbeqm,1519192456,NaN,"As long as you are living, you are winning.""""\..."


In [118]:
incels_reduced = incels[:100]
for comments in chats.keys():
    if comments in incels_reduced["parent"]:
        print("hey")